In [1]:
from __future__ import print_function, division
from scipy.optimize import fsolve #to get the initial value of vecp0
import numpy as np
from scikits.odes.sundials import ida
from scikits.odes import dae
import pandas as pd
import time
import cantera as ct
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
#from sympy import linsolve
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.family'] = 'Times New Roman'
colors = brewer2mpl.get_map('Set2','qualitative',8).mpl_colors

In [2]:
############################### initial conditions ##################################################################
#import the SiF4 + NH3 reaction mechnism
mech ='/Users/yuanjie/Dropbox/Cantera_intern/code/gas_surf_phase/gas_surf_bulk/mec.cti'
#import the models for gas and bulk
ct.suppress_thermo_warnings()
gas, bulk_Si, bulk_N = ct.import_phases(mech,['gas','SiBulk','NBulk',])

#import the model for gas-Si-N interface
gas_Si_N_interface = ct.Interface(mech, 'SI3N4',[gas,bulk_Si,bulk_N])
T = 1713 #K
p = 2 * ct.one_atm / 760.0 #Pa ~2Torr
#p = 2.632E-3*ct.one_atm 
gas.TPX = T,p,"NH3:6, SiF4:1"
bulk_Si.TP = T,p
bulk_N.TP = T,p
gas_Si_N_interface.TP = T,p
R = 8314 #universal gas constant
D = 5.08 * 10**-2 #diameter of the tube [m]
Ac = np.pi * D**2/4 # cross section of the tube [m]
Vdot = 588 * 10**-6/60 # volumetric rate [m^3/s]
mu = 5.7E-5 #kg/(m s) dynamic viscosity
perim = np.pi * D #perimeter of the tube
sigma_k = [2, 2, 2, 2, 4, 2] #site fraction of surface species as the same order of CTI file
#calculate the site fractions of surface species at the entrance of the tube at steady state
gas_Si_N_interface.advance_coverages(100.0)
Zk_0 = gas_Si_N_interface.coverages

######################################## IDA solver ###################################################################
def residual(z, vec, vecp, result):
    """ we create the residual equations for the problem
        vec = [u, rho, Yk, p, Zk]
    """
    gas.TP = T,vec[2+gas.n_species]
    gas.set_unnormalized_mass_fractions(vec[2:2+gas.n_species])
    bulk_Si.TP = T,vec[2+gas.n_species]
    bulk_N.TP = T,vec[2+gas.n_species]
    gas_Si_N_interface.TP = T,vec[2+gas.n_species]
    gas_Si_N_interface.set_unnormalized_coverages(vec[3+gas.n_species:])
    
    #mass continuity equation
    result[0] = vec[0]*vecp[1]+vec[1]*vecp[0]-perim*np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species]*gas.molecular_weights)/Ac
    #conservation of species
    for k in range(gas.n_species):
        result[1+k] = vec[1]*vec[0]*Ac*vecp[2+k] + vec[2+k]*perim*np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species]*gas.molecular_weights)\
                      - gas.net_production_rates[k]*gas.molecular_weights[k]*Ac\
                      - gas_Si_N_interface.net_production_rates[k]*gas.molecular_weights[k]*perim 
    #conservation of momentum
    result[1+gas.n_species] = 2*vec[1]*vec[0]*vecp[0] + np.power(vec[0],2)*vecp[1] + vecp[2+gas.n_species] + 32*vec[0]*mu/D**2 
    #equation of state
    result[2+gas.n_species] = vec[2+gas.n_species]*gas.mean_molecular_weight - vec[1]*R*T
    #algebraic constraints
    for j in range(gas_Si_N_interface.n_species):
        result[2+gas.n_species+j] = gas_Si_N_interface.net_production_rates[-gas_Si_N_interface.n_species+j]
    #replace the constraints with the condition sum(Zk) = 1 for the largest site fraction species
    index = np.argmax(gas_Si_N_interface.coverages)
    result[2+gas.n_species+index] = np.sum(gas_Si_N_interface.coverages) - 1

In [4]:
########use ling to solve the initial vecp###########
"""
   a = coefficient of [u', rho', Yk', P']
   b = RHS constant of each conservation equations
"""
rho0 = gas.density
u0 = 11.53 #m/s
################### a #########################
a = np.zeros((3+gas.n_species,3+gas.n_species))
a[0,:] = np.append([rho0,u0],np.zeros(1+gas.n_species))
for i in range(gas.n_species):
    a[1+i,2+i] = rho0*u0*Ac
a[1+gas.n_species,:] = np.append(np.append([2*rho0*u0, u0**2],np.zeros(gas.n_species)),[1])
coef = np.zeros(gas.n_species)
for j in range(gas.n_species):
    coef[j] = gas.P/gas.molecular_weights[j]/np.power(np.sum(gas.Y/gas.molecular_weights),2)
a[2+gas.n_species,:] = np.append(np.append([0,R*T],coef),-gas.mean_molecular_weight)
################### b ###########################
b = np.zeros(3+gas.n_species)
b[0] = perim*np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species]*gas.molecular_weights)/Ac
for i in range(gas.n_species):
    b[1+i] = gas.net_production_rates[i]*gas.molecular_weights[i]*Ac\
             + gas_Si_N_interface.net_production_rates[i]*gas.molecular_weights[i]*perim\
             - gas.Y[i]*perim*np.sum(gas_Si_N_interface.net_production_rates[:gas.n_species]*gas.molecular_weights) 
b[1+gas.n_species] = -32*u0*mu/D**2
b[2+gas.n_species] = 0
part_vecp0 = np.linalg.solve(a,b)
vecp0 = np.append(part_vecp0,np.zeros(gas_Si_N_interface.n_species))

In [6]:
solver = dae('ida', residual, 
             compute_initcond='yp0', #If yp0, then the differential variables (y of the ode system at time 0) will be used to solve for the derivatives of the differential variables, so yp0 will be calculated
             first_step_size=1e-18,
             atol=1e-6, #absolute tolerance for solution
             rtol=1e-6, #relative tolerance for solution
             algebraic_vars_idx=[np.arange(2+gas.n_species,2+gas.n_species+gas_Si_N_interface.n_species,1)], #If the given problem is of type DAE, some items of the residual
             #algebraic_vars_idx=[19, 20, 21, 22, 23, 24],
             #vector returned by the 'resfn' have to be treated as
                    #algebraic equations, and algebraic variables must be defined.
                    #These algebraic variables are denoted by the position (index)
                    #in the state vector y.
                    #All these indexes have to be specified in the
                    #'algebraic_vars_idx' array.
             #compute_initcond_t0 = 60,#When calculating the initial condition, specifies the time
                                      # until which the solver tries to
                                      #get the consistent values for either y0 or yp0 relative to
                                      #the starting time. Positive if t1 > t0, negative if t1 < t0
             max_steps=500000,
             old_api=False)#Forces use of old api (tuple of 7) if True or
                    #new api (namedtuple) if False.
                    #Other options may require new api, hence using this should
                    #be avoided if possible.

vec0 = np.append(np.append(np.append([11.53, gas.density], gas.Y),gas.P),Zk_0)
solution = solver.solve(np.arange(0,0.2,0.1), vec0,vecp0)
solution

SolverReturn(flag=-14, values=SolverVariables(t=None, y=None, ydot=None), errors=SolverVariables(t=0.0, y=array([  1.15300000e+01,   5.51650066e-04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   5.04595486e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         4.95404514e-01,   2.66644737e+02,   6.25700848e-02,
         9.15541628e-01,   3.14167985e-04,   2.08511788e-02,
         2.40980048e-04,   4.81960097e-04]), ydot=array([  4.94349247e+00,  -4.45614933e-04,  -4.58918273e-17,
         1.84080679e-06,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   2.92622057e-05,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         6.48667145e-01,   4.01383441e-09,  -6.52381823e-01,
         1.79751885e-08,   0.00000000e+00,   0.0000